<a href="https://colab.research.google.com/github/praffuln/agentic-ai/blob/master/langgraph_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: print hello world

print("hello world")


hello world


In [ ]:
!pip install google-generativeai tiktoken chromadb langgraph langchain langchain_community langchainhub ipykernel langchain_groq sentence_transformers


In [ ]:
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

## Setup Keys For GOOGLE_API_KEY


In [ ]:

# Assigning value to variable
GEMINI_API_KEY=''
SERPER_API_KEY = ''
GROQ_API_KEY = ''


# setup environments


In [ ]:
import os
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY




## setup LLM

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)


AI works by mimicking human intelligence processes through machines, particularly computer systems.  There's no single "how it works," as different AI approaches employ different techniques. However, several core concepts are common:

**1. Data as the Foundation:** AI systems learn from data.  The more data they're trained on, the better they generally perform. This data can be structured (like tables in a database) or unstructured (like text, images, or audio).

**2. Algorithms as the Engine:** Algorithms are sets of rules and statistical techniques that AI uses to analyze data, identify patterns, and make predictions or decisions.  These algorithms are the "brains" of the AI system, dictating how it processes information.  Different algorithms are suited for different tasks.

**3. Learning from Data:** This is the crucial aspect. AI systems don't just follow pre-programmed instructions; they learn from the data they are fed.  This learning can take several forms:

* **Supervised Lear

In [ ]:
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer # Import SentenceTransformer instead of GroqEmbedding


llm = ChatGroq(model="mixtral-8x7b-32768")

llm.invoke("tell me something about ai")

# Now use SentenceTransformer for embedding
embedding_model = SentenceTransformer('all-mpnet-base-v2')  # Choose a suitable pre-trained model
embedding = embedding_model.encode("tell me something about ai")

print(embedding) # Print the embedding to see the result

[-4.51203575e-03  2.20128950e-02 -4.33849730e-02 -1.77831128e-02
 -3.06594893e-02  8.90099071e-03  5.25341509e-03 -4.57331073e-03
  2.32293960e-02 -1.16360784e-02  1.13399606e-02  1.65772773e-02
 -1.06733488e-02  8.23313668e-02  3.30831483e-02 -2.61901524e-02
  2.75212415e-02 -4.16853130e-02  5.27629033e-02 -4.37602401e-02
 -4.88797277e-02 -2.66430527e-02 -5.22194691e-02 -2.14641169e-03
 -1.80472750e-02 -3.59376930e-02 -1.77159403e-02 -2.27067973e-02
 -1.07827326e-02 -7.24562630e-03 -3.39170173e-02 -9.26617160e-02
  3.07737361e-03  4.10415530e-02  1.43738237e-06 -4.15152535e-02
  3.65924835e-02  2.57706698e-02 -4.86444570e-02 -2.42777113e-02
 -9.24347248e-03  6.82267770e-02  3.60279693e-03  1.42892776e-02
 -5.68199642e-02  5.76578975e-02  5.50136119e-02  3.05756554e-02
 -1.68321887e-03  6.17104881e-02 -1.65109942e-03  4.90002614e-03
  3.29998769e-02 -2.69368626e-02  1.60680916e-02 -5.70929535e-02
  3.82944047e-02 -2.31349524e-02  6.24355376e-02 -5.50966188e-02
 -8.93084332e-03  4.00706

## Load Data

In [ ]:
urls= ["https://lilianweng.github.io/posts/2023-06-23-agent/",
      "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"]
docs = [WebBaseLoader(url).load() for url in urls]

docs[0][0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'title': "LLM Powered Autonomous Agents | Lil'Log",
 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\

In [ ]:
docs_list = [item for sublist in docs for item in sublist]

text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=5)
doc_splits=text_splitter.split_documents(docs_list)
doc_splits

In [ ]:

from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")


vectorstore=Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chrome",
    embedding=embedding_model
)


In [ ]:
retriever=vectorstore.as_retriever()


### convert retriever to tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.",

)

In [ ]:
tools=[retriever_tool]

## Fuctions & Graph

In [ ]:
def ai_assistant(state):
    print("---CALL AGENT---")
    messages = state['messages']
    print(f"this is my message: {messages}")

    llm_with_tool = llm.bind_tools(tools)
    response = llm_with_tool.invoke(messages)
    #response=handle_query(messages)
    return {"messages": [response]}



    # print("---CALL AGENT---")
    # messages = state['messages']
    # print(f"this is my message: {messages}")

    # if len(messages)>1:
    #     response=llm.invoke(messages[-1].content)
    #     return {"messages": [response]}
    # else:
    #     llm_with_tool = llm.bind_tools(tools)
    #     response = llm_with_tool.invoke(messages)
    #     #response=handle_query(messages)
    #     return {"messages": [response]}


In [ ]:
def retrieve(state):
  pass

In [ ]:
def rewrite(state):
  pass

In [ ]:
def generate(state):
  pass

In [ ]:
class AgentState(BaseModel):
    pass

In [ ]:
workflow=StateGraph(AgentState)
workflow.add_node("My_AI_Assistant",ai_assistant)
retrieve=ToolNode([retriever_tool])   ### way to add retriever_tool under node
workflow.add_node("retriever", retrieve)
workflow.add_node("Query_Rewriter", rewrite)
workflow.add_node("Output_Generator", generate)


In [ ]:
workflow.add_edge(START, "My_AI_Assistant")
workflow.add_edge("My_AI_Assistant", "Vector_Retriever")
# workflow.add_conditional_edges()

In [ ]:
app = workflow.compile()

In [ ]:
app.invoke({"messages": [HumanMessage(content="tell me something about ai")]})